# Pre Scan Data

This script will read in the text file stored in subject's func directory (`series_order_note.txt`) and will prep every run listed. 

`series_order_note.txt` should be a tab delimited text file. The first column should contain the series number for each scan. The second column should contain the name you would like to
assign to that run (e.g. TRAIN1). 

Each functional run will have the following steps applied:
1. reoriented to LAS
2. converted to nii.gz
3. disdaqs truncated off
4. header fixed to reflect the correct time units for the 4th dimension

**all of the original functional files will be moved into a new directory within the func directory called `originals`**

Each subject's anatomical will have the following steps applied:
1. reoriented to LAS
2. converted to nii.gz
3. skull stripped using BET

In [1]:
import sys
import os
import re
import numpy as np
import subprocess
import shutil

In [ ]:
subj = raw_input('Subject Number:')

# configuration vars
n_disdaqs = 8
run_key = 'series_order_note.txt'

# relevant paths
SRM_DIR = '/Volumes/adcock_lab/main/studies/CBT.01'
func_dir = os.path.join(SRM_DIR, 'data/func', subj)
anat_dir = os.path.join(SRM_DIR, 'data/anat', subj)

In [ ]:
# for each run in the key map
for run in runs:
    series_num = run[0]
    run_name = run[1]
    
    in_name = os.path.join(func_dir, ('bia5_' + subj + '_' + series_num + '_01.bxh'))
    out_name = os.path.join(func_dir, (run_name + '_temp.bxh'))
    
    # confirm the data orientation
    cmd_str = ('bxhreorient --orientation LAS ' + in_name + ' ' + out_name)
    os.system(cmd_str)
    
    # convert to nifti
    cmd_str = ('bxh2analyze --niigz ' + out_name + ' ' + os.path.join(func_dir, run_name))
    os.system(cmd_str)
    
    # retrieve the current number of timepoints
    p = subprocess.Popen(['fslval', os.path.join(func_dir, run_name), 'dim4'], stdout=subprocess.PIPE) 
    n_orig_timepts = p.stdout.readline().strip(' \n')
    
    # calculate how many timepts there will be after removing disdaqs
    n_timepts = int(n_orig_timepts) - n_disdaqs
    
    # remove disdaqs
    cmd_str = ('fslroi ' + os.path.join(func_dir, run_name) + ' ' + os.path.join(func_dir, run_name) + ' ' + str(n_disdaqs) + ' ' + str(n_timepts))
    os.system(cmd_str)
    
    # edit the header to fix the TR units (fslroi appears to have a bug which messes this up)
    cmd_str = ('fslhd -x ' + os.path.join(func_dir, run_name) + ' > ' + os.path.join(func_dir, 'old_hdr.txt'))        # write the header info to a text file
    os.system(cmd_str)
    old_header = open(os.path.join(func_dir, 'old_hdr.txt'), 'r')            # open the old header for reading
    new_header = open(os.path.join(func_dir, 'new_hdr.txt'), 'w')            # create a new header file to write into 
    for line in old_header:
        if re.search('time_units ', line):
            new_line = line.replace('8', '16')        # change the units from 8 (aka seconds) to 16 (aka ms)
        elif re.search('time_units_name', line):
            new_line = line.replace("'s'", "'ms'")        # change the unit label too (just for cleanliness)
        else:
            new_line = line
        new_header.write(new_line)
    new_header.close()
    old_header.close()
    
    # update the header info
    cmd_str = ('fslcreatehd ' + os.path.join(func_dir, 'new_hdr.txt') + ' ' + os.path.join(func_dir, run_name) + '.nii.gz')        # update the dataset with the new header
    os.system(cmd_str)
    
    # cleanup
    os.remove(os.path.join(func_dir, 'old_hdr.txt'))
    os.remove(os.path.join(func_dir, 'new_hdr.txt'))
    
    
# reformat the anatomical
for anat in ['full_anat']:

    series = ('bia5_' + subj + '_005.bxh')


    # confirm orientation
    cmd_str = ('bxhreorient --orientation LAS ' + os.path.join(anat_dir, series) + ' ' + os.path.join(anat_dir, (anat + '_temp.bxh')))
    os.system(cmd_str)

     #convert to nii.gz
    cmd_str = ('bxh2analyze --niigz ' + os.path.join(anat_dir, (anat + '_temp.bxh')) + ' ' + os.path.join(anat_dir, anat))
    os.system(cmd_str)
         
     #BET
    cmd_str = ('bet ' + os.path.join(anat_dir, (anat + '.nii.gz')) + ' ' + os.path.join(anat_dir, (anat + '_brain')) + ' -R -f 0.35 -g 0')
    os.system(cmd_str)

# clean up
cmd_str = ('rm ' + os.path.join(func_dir, '*_temp*')) 
os.system(cmd_str)
cmd_str = ('rm ' + os.path.join(anat_dir, '*_temp*')) 
os.system(cmd_str)

# move original files to a new directory
os.mkdir(os.path.join(func_dir,'originals'))
dirlist = os.listdir(func_dir)
for item in dirlist:
    if item[0]=='b':
        shutil.move(os.path.join(func_dir,item), os.path.join(func_dir,'originals'))